In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from cleanMDA import extractTable, divide_chunks, pullMDA, getXy
from text_analysis import text_analysis_wordall, text_analysis_sentenceall, get_polarity, low_subjectivity, model_analysis
from textblob import TextBlob
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Sentiment Analysis by Word

In [11]:
# company name and text in dataframe
df_sentences = pd.read_pickle('Cleaned_MDA_sentences.pkl')

In [3]:
# function to get GDP for 2007-2017 (import from GitHut)
def get_GDP_visual(df):
    USA = np.array(df[df['Data Source'] == 'United States'])

    gdp_USA = pd.DataFrame(USA, columns=['Country Name', 'Country COde', 'Indicator Name', 'Indicator Code',
                                                 '1960','1961','1962','1963','1964','1965','1966','1967','1968','1969',
                                                 '1970','1971','1972','1973','1974','1975','1976','1977','1978','1979',
                                                 '1980','1981','1982','1983','1984','1985','1986','1987','1988','1989',
                                                 '1990','1991','1992','1993','1994','1995','1996','1997','1998','1999',
                                                 '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009',
                                                 '2010','2011','2012','2013','2014','2015','2016','2017','2018'])

    gdp_USA = gdp_USA.drop(columns=['Country Name','Country COde','Indicator Name','Indicator Code','1960','1961','1962',
                                   '1963','1964','1965','1966','1967','1968','1969', '1970','1971','1972','1973','1974',
                                   '1975','1976','1977','1978','1979', '1980','1981','1982','1983','1984','1985','1986',
                                   '1987','1988','1989', '1990','1991','1992','1993','1994','1995','1996','1997','1998',
                                   '1999', '2000','2001','2002','2003','2004','2005','2006','2018'])

    gdp_USA_trans = gdp_USA.transpose()
    gdp_USA_trans = gdp_USA_trans.rename(columns={0:'Y'})
    
    return gdp_USA_trans

In [4]:
gdp_df = pd.read_csv('gdp_annual.csv')
gdp_USA = get_GDP_visual(gdp_df)

In [13]:
# call the function text_analysis_wordall on the MDA text df
analysis_df = text_analysis_wordall(df)
analysis_df

,X,y
2007,"[(frequently, 0.1, 0.3), (other, -0.125, 0.375...",1.77857
2008,"[(frequently, 0.1, 0.3), (other, -0.125, 0.375...",-0.291621
2009,"[(frequently, 0.1, 0.3), (other, -0.125, 0.375...",-2.77553
2010,"[(frequently, 0.1, 0.3), (other, -0.125, 0.375...",2.53192
2011,"[(frequently, 0.1, 0.3), (other, -0.125, 0.375...",1.60145
2012,"[(other, -0.125, 0.375), (certain, 0.214285714...",2.22403
2013,"[(other, -0.125, 0.375), (certain, 0.214285714...",1.67733
2014,"[(other, -0.125, 0.375), (certain, 0.214285714...",2.56919
2015,"[(other, -0.125, 0.375), (certain, 0.214285714...",2.86159
2016,"[(other, -0.125, 0.375), (certain, 0.214285714...",1.48528


In [6]:
def text_analysis_sentence(text, subjective=True):
    pos = []
    neg = []
    # split for each sentence
    for word in text.split(' . '):
        blob = TextBlob(word)
        sent = blob.sentiment
        # eliminate words with high subjectivity
#         if subjective:
#             if sent.subjectivity < 0.7:
#                 if sent.polarity > 0.25:
#                     pos.append(sent.polarity)
#                 elif sent.polarity < -0.25:
#                     neg.append(sent.polarity)
        
    return word,sent.polarity,sent.subjectivity

In [7]:
## Sentiment Analysis by Sentence